# Challenge Description

Company XYZ is a worldwide e-commerce site with localized versions of the site.

A data scientist at XYZ noticed that Spain-based users have a much higher conversion rate than any other Spanish-speaking country.

Spain and LatAm country manager suggested that one reason could be translation. All Spanish-speaking countries had the same translation of the site which was written by a Spaniard. Therefore, they agreed to try a test where each country would have its own translation written by a local. That is, Argentinian users would see a translation written by an Argentinian, Mexican users written by a Mexican and so on. Obviously, nothing would change for users from Spain.

After they run the test however, they are really surprised because the test is negative. That is, it appears that the non-highly localized translation was doing better!

You are asked to:
- Confirm that test is actually negative. I.e., the old version of the site with just one translation across Spain and LatAm performs better

- Explain why that might be happening. Are the localized translations really worse?
- If you identified what was wrong, design an algorithm that would return FALSE if the same problem is happening in the future and TRUE if everything is good and results can be trusted.


# Data Description

## test_table
General information about the test results

Columns:
- **user_id:** the id of the user. Unique by user. Can be joined to user id in the other table. For each user, we just check whether conversion happens the first time they land on the site since the test started.
- **date:** when they came to the site for the first time since the test started
- **source:** marketing channel: Ads, SEO, Direct. Direct means everything except for ads and SEO, such as directly typing site URL on the browser, downloading the app w/o coming from SEO or Ads, referral friend, etc.
- **device:** device used by the user. It can be mobile or web
- **browser_language:** in browser or app settings, the language chosen by the user. It can be EN, ES, Other (Other means any language except for English and Spanish)
- **ads_channel:** if marketing channel is ads, this is the site where the ad was displayed. It can be: Google, Facebook, Bing, Yahoo ,Other. If the user didn’t come via an ad, this field is NA
- **browser:** user browser. It can be: IE, Chrome, Android_App, FireFox, Iphone_App, Safari, Opera
- **conversion:** whether the user converted (1) or not (0). This is our label. A test is considered successful if it increases the proportion of users who convert.
- **test:** users are randomly split into test (1) and control (0). Test users see the new translation and control the old one. For Spain-based users, this is obviously always 0 since there is no change there.

---

## user_table
Some information about the user

Columns:
- **user_id:** the id of the user. It can be joined to user id in the other table
- **sex:** user sex: Male or Female
- **age:** user age (self-reported)
- **country:** user country based on ip address

# Initial Exploration

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind

In [ ]:
df_tt = pd.read_csv('PRIVATE CSV')
df_ut = pd.read_csv('PRIVATE CSV')

In [ ]:
print(df_tt.shape)
df_tt.head()

(453321, 9)


,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


In [ ]:
print(df_tt.shape)
df_ut.head()

(453321, 9)


,user_id,sex,age,country
0,765821,M,20,Mexico
1,343561,F,27,Nicaragua
2,118744,M,23,Colombia
3,987753,F,27,Venezuela
4,554597,F,20,Spain


In [ ]:
print(df_tt.isnull().sum(),'\n')
print(df_tt.info())

user_id                  0
date                     0
source                   0
device                   0
browser_language         0
ads_channel         271444
browser                  0
conversion               0
test                     0
dtype: int64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453321 entries, 0 to 453320
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           453321 non-null  int64 
 1   date              453321 non-null  object
 2   source            453321 non-null  object
 3   device            453321 non-null  object
 4   browser_language  453321 non-null  object
 5   ads_channel       181877 non-null  object
 6   browser           453321 non-null  object
 7   conversion        453321 non-null  int64 
 8   test              453321 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 31.1+ MB
None


In [ ]:
print(df_ut.isnull().sum(), '\n')
print(df_ut.info())

user_id    0
sex        0
age        0
country    0
dtype: int64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452867 entries, 0 to 452866
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  452867 non-null  int64 
 1   sex      452867 non-null  object
 2   age      452867 non-null  int64 
 3   country  452867 non-null  object
dtypes: int64(2), object(2)
memory usage: 13.8+ MB
None


`.info()` showed the `user_id` count in the two tables don't match. Double check below. With such a small difference we'll go ahead and drop them in the merge but the company should take a look at why there is a discrepancy later.

In [ ]:
print('user table:', df_ut['user_id'].count(), df_ut['user_id'].nunique())
print('test table:', df_tt['user_id'].count(), df_tt['user_id'].nunique())

user table: 452867 452867
test table: 453321 453321


In [ ]:
df = df_ut.merge(df_tt, on='user_id')

print(df.shape)
df.head()

(452867, 12)


,user_id,sex,age,country,date,source,device,browser_language,ads_channel,browser,conversion,test
0,765821,M,20,Mexico,2015-12-02,Ads,Mobile,ES,Yahoo,Android_App,0,1
1,343561,F,27,Nicaragua,2015-12-04,Ads,Web,ES,Facebook,Safari,0,0
2,118744,M,23,Colombia,2015-11-30,Ads,Mobile,ES,Facebook,Android_App,0,1
3,987753,F,27,Venezuela,2015-12-04,SEO,Web,ES,NaN,IE,0,1
4,554597,F,20,Spain,2015-12-04,Direct,Web,ES,NaN,Chrome,0,0


# Testing and Exploration

Verify the initial claim that kicked all this off, does Spain have an abnormally high conversion rate? Yep. Also, Argentina and Uruguay have abnormally low conversion rates as well.

In [ ]:
df.groupby('country')['conversion'].mean().sort_values(ascending=False)

country
Spain          0.079719
Costa Rica     0.053494
Nicaragua      0.053399
Colombia       0.051332
El Salvador    0.050765
Mexico         0.050341
Peru           0.050258
Chile          0.049704
Venezuela      0.049666
Guatemala      0.049653
Honduras       0.049253
Ecuador        0.049072
Paraguay       0.048863
Bolivia        0.048634
Panama         0.048089
Argentina      0.013994
Uruguay        0.012821
Name: conversion, dtype: float64

Verify the test. Significant difference in the conversion rate between the control group and the test group. Verified by the t-test, the test group performed worse than leaving things the same.

In [ ]:
df[df['test'] == 1]['conversion'].mean(), df[df['test'] == 0]['conversion'].mean()

(0.043411161678422794, 0.05515557186420519)

In [ ]:
test_results =  ttest_ind(df[(df['test'] == 1) & (df['country'] != 'Spain')]['conversion'],
                            df[(df['test'] == 0) & (df['country'] != 'Spain')]['conversion'],
                            equal_var=False)

print('P-Value:', test_results.pvalue)
print('T-Stat:', test_results.statistic)

P-Value: 1.928917857779903e-13
T-Stat: -7.353895203080277


It seems unlikely that localized sites would perform worse, especially by so much, so we need to look for potential causes.

We have almost half a million observations so it is unlikely that our sample size is too small. The next likely culprit is that the test and control groups were not split evenly on one or more variables. Ideally, our control and test groups should only differ on one variable.

In [ ]:
df_one_hot = pd.get_dummies(df)

In [ ]:
for col in df_one_hot.columns:
    print(col)
    print(df_one_hot.groupby('test')[col].mean(), '\n')

user_id
test
0    500364.345278
1    499483.813462
Name: user_id, dtype: float64 

age
test
0    27.144850
1    27.115236
Name: age, dtype: float64 

conversion
test
0    0.055156
1    0.043411
Name: conversion, dtype: float64 

test
test
0    0
1    1
Name: test, dtype: int64 

sex_F
test
0    0.415335
1    0.416681
Name: sex_F, dtype: float64 

sex_M
test
0    0.584665
1    0.583319
Name: sex_M, dtype: float64 

country_Argentina
test
0    0.039461
1    0.173223
Name: country_Argentina, dtype: float64 

country_Bolivia
test
0    0.023409
1    0.025833
Name: country_Bolivia, dtype: float64 

country_Chile
test
0    0.041558
1    0.045807
Name: country_Chile, dtype: float64 

country_Colombia
test
0    0.114251
1    0.125001
Name: country_Colombia, dtype: float64 

country_Costa Rica
test
0    0.011219
1    0.012277
Name: country_Costa Rica, dtype: float64 

country_Ecuador
test
0    0.033894
1    0.036422
Name: country_Ecuador, dtype: float64 

country_El Salvador
test
0    0.017327
1

Looking at each of the variables, we see that Uruguay and Argentina are the culprits. There seems a relatively large difference of the mean between the test and control groups for those two countries. We can test this further by removing those two countries from the A/B test and seeing if we get the test results we expected.

In [ ]:
df_one_hot.groupby('test')[['country_Argentina', 'country_Uruguay']].mean()

,country_Argentina,country_Uruguay
test,,
0,0.039461,0.001750
1,0.173223,0.017236


Remove Uruguay and Argentina to remove the irregularity and still removing Spain since it doesn't change. Run the test and we see a non-significant result. We found and confirmed the problem but its a bit underwhelming when we were trying to improve conversion rate.

In [ ]:
df_correction = df[(df['country'] != 'Argentina') & 
                   (df['country'] != 'Uruguay') & 
                   (df['country'] != 'Spain')]

In [ ]:
ttest_ind(df_correction[df_correction['test'] == 1]['conversion'],
                        df_correction[df_correction['test'] == 0]['conversion'],
                        equal_var=False)

Ttest_indResult(statistic=0.35834557944814466, pvalue=0.7200849282884503)

Per the final prompt, create a function to catch this issue in the future.

In [ ]:
def verify_distributions(df_one_hot):
    for col in df_one_hot.columns:
        splits = df_one_hot.groupby('test')[col].mean()
        percent_diff = (1-(splits[0]/splits[1]))
        if percent_diff > .1:
            return False
    return True